In [42]:
# import libraries

import pandas as pd 
import numpy as np 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

import requests 
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json # If reading in data from Json file
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # maps
from geopy.geocoders import Nominatim # uses address to get lat/long

# Installations
#!conda install -c conda-forge geopy 
#!conda install -c conda-forge folium=0.5.0 

#  Data Acquisition

#### Scrape Wikipedia for Names of Neighborhoods in Austin

In [43]:
# scrape list of Austin neighborhoods

neighborhoods = []

url = 'https://en.wikipedia.org/wiki/List_of_Austin_neighborhoods'
res = requests.get(url)
if res.status_code == 200:
    soup = BeautifulSoup(res.content, 'lxml')
    div = soup.find_all('div', {'style': 'padding:0em 0.25em'})[13]
    for li in div.find_all('li'):
        name = li.text
        neighborhoods.append(name)

df = pd.DataFrame({'neighborhood': neighborhoods})  
df.head()

,neighborhood
0,Allandale
1,Anderson Mill
2,Barton Hills
3,Blackland
4,Bouldin Creek


Here I am editing the name of some of the neighborhoods to match what is in the neighborhoods.com sales data to test if geopy can locate coordinates based on the name change. I previously ran geopy with the original neighborhood name from the Wikipedia scrape which did not return results.

In [44]:
# Rename neighborhoods for searchability
df.loc[26,'neighborhood'] = 'Hyde Park'
df.loc[30,'neighborhood'] = 'Mueller'
df.loc[31,'neighborhood'] = 'North Burnet'

In [45]:
# Create columns with 'neighborhood' + 'Austin, Texas' as location to search for lat/long

locations = []

for name in df['neighborhood']:
    location = name + ', Austin, Texas'
    locations.append(location)

df['location'] = locations
df.head()

,neighborhood,location
0,Allandale,"Allandale, Austin, Texas"
1,Anderson Mill,"Anderson Mill, Austin, Texas"
2,Barton Hills,"Barton Hills, Austin, Texas"
3,Blackland,"Blackland, Austin, Texas"
4,Bouldin Creek,"Bouldin Creek, Austin, Texas"


In [46]:
# Check for nulls
df.isnull().sum()

neighborhood    0
location        0
dtype: int64

In [47]:
# View shape
df.shape

(59, 2)

#### Obtain Lat/Long Data

This code uses try/except to account for latitudes and longitudes that cannot be found with the geopy library. This is a limitation of using this package. Geopy documentation can be found here: https://geopy.readthedocs.io/en/stable/

In [48]:
# Look up lat/long with geopy library

lats = []
longs = []
    
for name in df['neighborhood']:
    neighborhood = name + ', Austin, Texas'
    geolocator = Nominatim(user_agent = 'foursquare_agent')
    try:
        location = geolocator.geocode(neighborhood)
        lat = location.latitude
        long = location.longitude 
        lats.append(lat)
        longs.append(long)
    except AttributeError: 
        print(f'Problem with {name} data or cannot Geocode')

Problem with Bremond Block Historic District data or cannot Geocode
Problem with Moore's Crossing Historic District data or cannot Geocode
Problem with Shadow Lawn Historic District data or cannot Geocode
Problem with Spyglass-Barton's Bluff data or cannot Geocode
Problem with Willow-Spence Streets data or cannot Geocode


In [49]:
# Drop locations cannot obtain lat/long for and do not have sales data for

drop_list = ['Bremond Block Historic District', "Moore's Crossing Historic District", 'Shadow Lawn Historic District', "Spyglass-Barton's Bluff", 'Willow-Spence Streets']

for item in drop_list:
    df.drop(df[df['neighborhood'] == item].index, inplace = True) 

Because I will be merging dataframes without a common column name I am resetting the index wherever necessary to be able to merge on the index.

In [50]:
# Reset index
df.reset_index(drop='True', inplace=True)

In [51]:
# Populate df with lat/long

df['latitude'] = lats
df['longitude'] = longs
df.tail()

,neighborhood,location,latitude,longitude
49,West Line,"West Line, Austin, Texas",30.424179,-97.739170
50,Windsor Hills,"Windsor Hills, Austin, Texas",30.362072,-97.679214
51,Windsor Park,"Windsor Park, Austin, Texas",30.313549,-97.691095
52,Wooten,"Wooten, Austin, Texas",30.360467,-97.723320
53,Zilker,"Zilker, Austin, Texas",30.254236,-97.769581


When viewing the plotted coordinates on a map, it was clear that some of the coordinates were wrong since they were far outside of Austin.  I used Google Earth to check the coordinates generated by geopy and found following locations that need to be updated.

In [52]:
# Fix bad coordinates

# Canyon Creek
df.loc[df['neighborhood'] == 'Canyon Creek', ['latitude']] = 30.434033
df.loc[df['neighborhood'] == 'Canyon Creek', ['longitude']] = -97.820708

# Cicle C Ranch
df.loc[df['neighborhood'] == 'Circle C Ranch', ['latitude']] = 30.190339
df.loc[df['neighborhood'] == 'Circle C Ranch', ['longitude']] = -97.88905

# Easton Park
df.loc[df['neighborhood'] == 'Easton Park', ['latitude']] = 30.155247
df.loc[df['neighborhood'] == 'Easton Park', ['longitude']] = -97.718764

# Jollyville
df.loc[df['neighborhood'] == 'Jollyville', ['latitude']] = 30.462108
df.loc[df['neighborhood'] == 'Jollyville', ['longitude']] = -97.737042

# Oak Hill
df.loc[df['neighborhood'] == 'Oak Hill', ['latitude']] = 30.231431
df.loc[df['neighborhood'] == 'Oak Hill', ['longitude']] = -97.865314

# Onion Creek
df.loc[df['neighborhood'] == 'Onion Creek', ['latitude']] = 30.140056
df.loc[df['neighborhood'] == 'Onion Creek', ['longitude']] = -97.781803

# Sixth Street
df.loc[df['neighborhood'] == 'Sixth Street', ['latitude']] = 30.266225
df.loc[df['neighborhood'] == 'Sixth Street', ['longitude']] = -97.738044

# Swedish Hill
df.loc[df['neighborhood'] == 'Swedish Hill', ['latitude']] = 30.272428
df.loc[df['neighborhood'] == 'Swedish Hill', ['longitude']] = -97.756531

In [53]:
# Export for Tableau
df.to_excel("../data/lat_long.xlsx", index=False)

The website uses a specific format for the URL for each neighborhood so I am generating the URL for each neighborhood to get the sales data. 

In [54]:
# Create column for url format to generate url
df['url_format'] = [item.replace(' ','-').lower() for item in df['neighborhood']]

In [55]:
# Preview
df.head()

,neighborhood,location,latitude,longitude,url_format
0,Allandale,"Allandale, Austin, Texas",30.339309,-97.746972,allandale
1,Anderson Mill,"Anderson Mill, Austin, Texas",30.455835,-97.807096,anderson-mill
2,Barton Hills,"Barton Hills, Austin, Texas",30.251571,-97.784106,barton-hills
3,Blackland,"Blackland, Austin, Texas",30.281491,-97.720924,blackland
4,Bouldin Creek,"Bouldin Creek, Austin, Texas",30.255667,-97.755481,bouldin-creek


In [56]:
# Create urls

urls = []

for name in df['url_format']:
    url = 'https://www.neighborhoods.com/' + name + '-austin-tx'
    urls.append(url)

df['url'] = urls
df.head()

,neighborhood,location,latitude,longitude,url_format,url
0,Allandale,"Allandale, Austin, Texas",30.339309,-97.746972,allandale,https://www.neighborhoods.com/allandale-austin-tx
1,Anderson Mill,"Anderson Mill, Austin, Texas",30.455835,-97.807096,anderson-mill,https://www.neighborhoods.com/anderson-mill-au...
2,Barton Hills,"Barton Hills, Austin, Texas",30.251571,-97.784106,barton-hills,https://www.neighborhoods.com/barton-hills-aus...
3,Blackland,"Blackland, Austin, Texas",30.281491,-97.720924,blackland,https://www.neighborhoods.com/blackland-austin-tx
4,Bouldin Creek,"Bouldin Creek, Austin, Texas",30.255667,-97.755481,bouldin-creek,https://www.neighborhoods.com/bouldin-creek-au...


In [57]:
# View count
df.shape

(54, 6)

#### Sales Data

In [58]:
# Pull median and average prices

avg_prices = []
med_prices = []
urls = []

for url in df['url']:
    try:
        res = requests.get(url)
        if res.status_code == 200:
            soup = BeautifulSoup(res.content, 'lxml')
            div = soup.find('div', {'class':'detail-block-2'})
            for li in div.find_all('li')[2]:
                med_price = li.text
                med_prices.append(med_price)
                urls.append(url)
            for li in div.find_all('li')[3]:
                avg_price = li.text
                avg_prices.append(avg_price)
    except: 
        print(f"{url} does not exist or cannot locate data")

https://www.neighborhoods.com/montopolis-austin-tx does not exist or cannot locate data
https://www.neighborhoods.com/northwest-hills-austin-tx does not exist or cannot locate data


In [59]:
# Create df for prices

prices_df = pd.DataFrame({'median_prices': med_prices, 'avg_price_per_sqft': avg_prices, 'url': urls})

In [60]:
# Preview prices_df
prices_df.head()

,median_prices,avg_price_per_sqft,url
0,Median Sale Price:,Average $ per sq ft:,https://www.neighborhoods.com/allandale-austin-tx
1,"$591,000",$350,https://www.neighborhoods.com/allandale-austin-tx
2,Median Sale Price:,Average $ per sq ft:,https://www.neighborhoods.com/anderson-mill-au...
3,"$321,495",$194,https://www.neighborhoods.com/anderson-mill-au...
4,Median Sale Price:,Average $ per sq ft:,https://www.neighborhoods.com/barton-hills-aus...


In [61]:
# Drop extra rows
prices_df.drop(prices_df[prices_df['median_prices'] == 'Median Sale Price:'].index, inplace = True) 

In [62]:
# Reset index after drop
prices_df.reset_index(drop='True', inplace=True)

In [63]:
# View shape
print(prices_df.shape)
print(df.shape)

(39, 3)
(54, 6)


In [64]:
# Drop location cannot obtain sales data for

drop_list = ['https://www.neighborhoods.com/montopolis-austin-tx', 'https://www.neighborhoods.com/northwest-hills-austin-tx']

for item in drop_list:
    df.drop(df[df['url'] == item].index, inplace = True) 

In [65]:
no_connection = []

for url in df['url']:
    try:
        res = requests.get(url)
        if res.status_code == 404:
            print(f'{url} does not exist')
            no_connection.append(url)
    except: 
        print(f"{url} does not exist or cannot locate data")

https://www.neighborhoods.com/blackland-austin-tx does not exist
https://www.neighborhoods.com/chinatown-austin-tx does not exist
https://www.neighborhoods.com/congress-avenue-austin-tx does not exist
https://www.neighborhoods.com/the-drag-austin-tx does not exist
https://www.neighborhoods.com/east-riverside-oltorf-austin-tx does not exist
https://www.neighborhoods.com/french-place-austin-tx does not exist
https://www.neighborhoods.com/jollyville-austin-tx does not exist
https://www.neighborhoods.com/rainey-street-austin-tx does not exist
https://www.neighborhoods.com/saint-john-austin-tx does not exist
https://www.neighborhoods.com/sixth-street-austin-tx does not exist
https://www.neighborhoods.com/swedish-hill-austin-tx does not exist
https://www.neighborhoods.com/waters-park-austin-tx does not exist
https://www.neighborhoods.com/west-line-austin-tx does not exist


In [66]:
# Drop locations with 404 errors

for item in no_connection:
    df.drop(df[df['url'] == item].index, inplace = True) 

In [67]:
# Reset index after drop
df.reset_index(drop='True', inplace=True)

In [68]:
# View shape
print(prices_df.shape)
print(df.shape)

(39, 3)
(39, 6)


In [69]:
# Combine dataframes
data_df = df.merge(prices_df, on='url',  how='left')
data_df.head()

,neighborhood,location,latitude,longitude,url_format,url,median_prices,avg_price_per_sqft
0,Allandale,"Allandale, Austin, Texas",30.339309,-97.746972,allandale,https://www.neighborhoods.com/allandale-austin-tx,"$591,000",$350
1,Anderson Mill,"Anderson Mill, Austin, Texas",30.455835,-97.807096,anderson-mill,https://www.neighborhoods.com/anderson-mill-au...,"$321,495",$194
2,Barton Hills,"Barton Hills, Austin, Texas",30.251571,-97.784106,barton-hills,https://www.neighborhoods.com/barton-hills-aus...,"$667,750",$487
3,Bouldin Creek,"Bouldin Creek, Austin, Texas",30.255667,-97.755481,bouldin-creek,https://www.neighborhoods.com/bouldin-creek-au...,"$725,750",$546
4,Brentwood,"Brentwood, Austin, Texas",30.331264,-97.736465,brentwood,https://www.neighborhoods.com/brentwood-austin-tx,"$495,250",$370


In [70]:
# Check dtypes
data_df.dtypes

neighborhood           object
location               object
latitude              float64
longitude             float64
url_format             object
url                    object
median_prices          object
avg_price_per_sqft     object
dtype: object

In [71]:
# Format price

data_df['median_prices'] = [item.replace('$','').replace(',','') for item in data_df['median_prices']]
data_df.head()

,neighborhood,location,latitude,longitude,url_format,url,median_prices,avg_price_per_sqft
0,Allandale,"Allandale, Austin, Texas",30.339309,-97.746972,allandale,https://www.neighborhoods.com/allandale-austin-tx,591000,$350
1,Anderson Mill,"Anderson Mill, Austin, Texas",30.455835,-97.807096,anderson-mill,https://www.neighborhoods.com/anderson-mill-au...,321495,$194
2,Barton Hills,"Barton Hills, Austin, Texas",30.251571,-97.784106,barton-hills,https://www.neighborhoods.com/barton-hills-aus...,667750,$487
3,Bouldin Creek,"Bouldin Creek, Austin, Texas",30.255667,-97.755481,bouldin-creek,https://www.neighborhoods.com/bouldin-creek-au...,725750,$546
4,Brentwood,"Brentwood, Austin, Texas",30.331264,-97.736465,brentwood,https://www.neighborhoods.com/brentwood-austin-tx,495250,$370


In [72]:
# Format price

data_df['avg_price_per_sqft'] = [item.replace('$','') for item in data_df['avg_price_per_sqft']]
data_df.head()

,neighborhood,location,latitude,longitude,url_format,url,median_prices,avg_price_per_sqft
0,Allandale,"Allandale, Austin, Texas",30.339309,-97.746972,allandale,https://www.neighborhoods.com/allandale-austin-tx,591000,350
1,Anderson Mill,"Anderson Mill, Austin, Texas",30.455835,-97.807096,anderson-mill,https://www.neighborhoods.com/anderson-mill-au...,321495,194
2,Barton Hills,"Barton Hills, Austin, Texas",30.251571,-97.784106,barton-hills,https://www.neighborhoods.com/barton-hills-aus...,667750,487
3,Bouldin Creek,"Bouldin Creek, Austin, Texas",30.255667,-97.755481,bouldin-creek,https://www.neighborhoods.com/bouldin-creek-au...,725750,546
4,Brentwood,"Brentwood, Austin, Texas",30.331264,-97.736465,brentwood,https://www.neighborhoods.com/brentwood-austin-tx,495250,370


In [73]:
# Convert dtype to float

data_df['median_prices'] = data_df['median_prices'].astype(float)

In [74]:
# Convert dtype to float

data_df['avg_price_per_sqft'] = data_df['avg_price_per_sqft'].astype(float)

In [75]:
# Confirm dtype conversion

data_df.dtypes

neighborhood           object
location               object
latitude              float64
longitude             float64
url_format             object
url                    object
median_prices         float64
avg_price_per_sqft    float64
dtype: object

In [76]:
# Drop rows with no Foursquare data

drop_items = ['Allandale', 'Wooten']

for item in drop_items:
    data_df.drop(data_df[data_df['neighborhood'] == item].index, inplace = True) 

In [77]:
# Save data_df 
data_df.to_csv('../data/data_df.csv', index=False)

In [78]:
# Check for bad coorindates

map_df = folium.Map(location=[data_df.iloc[0,2], data_df.iloc[0,3]], zoom_start=10)

# add markers to map
for lat, lng, name in zip(data_df['latitude'], data_df['longitude'], data_df['neighborhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_df)  
    
map_df

#### Foursquare Data

In [ ]:
# Establish Foursquare credentials for reference

CLIENT_ID = 'S5JB2KFD1NTX3EQLZEUCB5OLMPQTFOITAQG1ZMHVNB12UFH0' 
CLIENT_SECRET = '2SV2BAJSKBTEXPH41FHT2X3P1M1VQKYLZ54213I0VAZU2JPT' 
VERSION = '20200609' # Foursquare API version


In [ ]:
# Collect venue results from Foursquare
# Code adapted from IBM data science cert lab

radius = 1000 # meters 
limit = 200

venues = []

for lat, long, neighborhood in zip(data_df['latitude'], data_df['longitude'], data_df['neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        limit)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
#  Create dataframe for venues results
venues_df = pd.DataFrame(venues)
venues_df.columns = ['neighborhood', 'latitude', 'longitude', 'venue_name', 'venue_latitude', 'venue_longitude', 'venue_type']

# check the dataframe
print('Shape: {}'.format(venues_df.shape))
print('Unique venue types: {}'.format(len(venues_df['venue_type'].unique())))
venues_df.head()

In [ ]:
# Save dataframe with Foursquare data 
venues_df.to_csv('../data/venues_df.csv', index=False)